## Q-4.
Imagine you working as a sale manager now you need to predict the Revenue
and whether that particular revenue is on the weekend or not and find the
Informational_Duration using the Ensemble learning algorithm
Dataset This is the Dataset You can use this dataset for this question.

In [29]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('online_shoppers_intention.csv')
data

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.000000,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.000000,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.000000,0.0,Feb,3,3,1,4,Returning_Visitor,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3,145.0,0,0.0,53,1783.791667,0.007143,0.029031,12.241717,0.0,Dec,4,6,1,1,Returning_Visitor,True,False
12326,0,0.0,0,0.0,5,465.750000,0.000000,0.021333,0.000000,0.0,Nov,3,2,1,8,Returning_Visitor,True,False
12327,0,0.0,0,0.0,6,184.250000,0.083333,0.086667,0.000000,0.0,Nov,3,2,1,13,Returning_Visitor,True,False
12328,4,75.0,0,0.0,15,346.000000,0.000000,0.021053,0.000000,0.0,Nov,2,2,3,11,Returning_Visitor,False,False


In [3]:
data['VisitorType'].value_counts()

Returning_Visitor    10551
New_Visitor           1694
Other                   85
Name: VisitorType, dtype: int64

In [4]:
data['Month'].value_counts()

May     3364
Nov     2998
Mar     1907
Dec     1727
Oct      549
Sep      448
Aug      433
Jul      432
June     288
Feb      184
Name: Month, dtype: int64

In [5]:
data.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [7]:
data['Weekend'].astype("category")

0        False
1        False
2        False
3        False
4         True
         ...  
12325     True
12326     True
12327     True
12328    False
12329     True
Name: Weekend, Length: 12330, dtype: category
Categories (2, bool): [False, True]

In [8]:
def get_cleaned(df):
    return df[(df.Administrative <10) & (df.Administrative<150) & (df.ProductRelated<120)]

In [9]:
df_eda = data[data['VisitorType']!="Other"]


In [10]:
df_eda = get_cleaned(df_eda)

In [11]:
df_eda.shape

(11324, 18)

In [12]:
train_ratio = 0.8

In [13]:
num_samples = df_eda.shape[0]
all_samples_idxs = list(range(num_samples))
train_samples_idxs = list(range(int(num_samples*0.8)))
test_samples_idxs = list(range(train_samples_idxs[-1]+1, num_samples))
df_train = df_eda.iloc[train_samples_idxs]

df_test = df_eda.iloc[test_samples_idxs]
list_features = [col for col in list(df_eda.columns) if col!="Weekend"]
target_feature = "Weekend"

targetlabelencoder = preprocessing.LabelEncoder()

X_train_overall = df_train[list_features]
Y_train_overall = df_train[target_feature]

X_test_overall = df_test[list_features]
Y_test_overall = df_test[target_feature]


In [18]:
def pre_process(input_df):
    le = preprocessing.LabelEncoder()
    input_df['Month'] = le.fit_transform(input_df.Month.values)
    input_df['VisitorType'] = le.fit_transform(input_df.VisitorType.values)
    input_df['Revenue'] = le.fit_transform(input_df.Revenue.values)
    return input_df

In [19]:
# Define the number of folds
num_folds = 5

# Create an instance of KFold with the desired number of folds
kf = KFold(n_splits=num_folds)
for num_estimator in [10, 50, 100]:
    rfModel = RandomForestClassifier(n_estimators = num_estimator)
    avg_acc = 0
    fold_num = 1
    for train_index, test_index in kf.split(X_train_overall):
        X_train, X_test = pre_process(X_train_overall.iloc[train_index]), pre_process(X_train_overall.iloc[test_index])
        y_train, y_test = Y_train_overall.iloc[train_index], Y_train_overall.iloc[test_index]
        rfModel.fit(X_train, y_train)
        y_pred = rfModel.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        avg_acc += accuracy
        print(f"For fold_num = {fold_num} rf_model_param: num_estimator={num_estimator}, accuracy is {accuracy}")
        fold_num+=1
    print(f"Avg accuracy = {avg_acc/num_folds}")


C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['Month'] = le.fit_transform(input_df.Month.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['VisitorType'] = le.fit_transform(input_df.VisitorType.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

For fold_num = 1 rf_model_param: num_estimator=10, accuracy is 0.7428256070640177


C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['Month'] = le.fit_transform(input_df.Month.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['VisitorType'] = le.fit_transform(input_df.VisitorType.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

For fold_num = 2 rf_model_param: num_estimator=10, accuracy is 0.8079470198675497


C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['Month'] = le.fit_transform(input_df.Month.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['VisitorType'] = le.fit_transform(input_df.VisitorType.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

...

For fold_num = 3 rf_model_param: num_estimator=100, accuracy is 0.8233995584988962


C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['Month'] = le.fit_transform(input_df.Month.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['VisitorType'] = le.fit_transform(input_df.VisitorType.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

For fold_num = 4 rf_model_param: num_estimator=100, accuracy is 0.7687637969094923


C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['Month'] = le.fit_transform(input_df.Month.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['VisitorType'] = le.fit_transform(input_df.VisitorType.values)
C:\Users\dell\AppData\Local\Temp\ipykernel_12132\162190570.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

For fold_num = 5 rf_model_param: num_estimator=100, accuracy is 0.7476532302595251
Avg accuracy = 0.7794423458311543


In [20]:
# prediction for whether revenue is on weekend or not
y_pred = rfModel.predict(X_test)

y_pred

array([False, False, False, ..., False, False, False])

In [27]:
def pre_process(input_df):
    le = preprocessing.LabelEncoder()
    input_df['Month'] = le.fit_transform(input_df.Month.values)
    input_df['VisitorType'] = le.fit_transform(input_df.VisitorType.values)
    input_df['Revenue'] = le.fit_transform(input_df.Revenue.values)
    input_df['Weekend'] = le.fit_transform(input_df.Weekend.values)
    
    return input_df

In [33]:
# Split the dataset into features (X) and target (y)
X = pre_process(data.drop('Informational_Duration', axis=1))
y = data['Informational_Duration']

In [34]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# RandomForestRegressor 
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)

# prediction for Informational_Duration
rf_predictions = dt_clf.predict(X_test)
rf_predictions

array([  0.        , 236.88785   ,   0.        , ...,  70.33516667,
       431.72748811,   0.        ])